Using historic data
using public sentiment - sentiment analysis
using XGBoost model

!pip install tensorflow-metal
%env TF_METAL_ENABLED=1
!pip install --upgrade tensorflow-metal

In [1]:
#

# Imports
import sys
import os
import platform
from os.path import exists
from watermark import watermark

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

#

In [2]:
# Report Technologies
print(f'Python Platform: {platform.platform()}')
print(f'Python {sys.version}')
print(watermark())
print(watermark(iversions=True, globals_=globals()))

mps_available = tf.config.list_logical_devices("MPS")
device = "/device:MPS:0" if len(mps_available) > 0 else "/device:CPU:0"
print(f"TensorFlow version: {tf.__version__}\nIs MPS available?: {len(mps_available) > 0}\nUsing device: {device}")

Python Platform: macOS-13.0.1-arm64-arm-64bit
Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
Last updated: 2023-05-09T15:18:58.691185-05:00

Python implementation: CPython
Python version       : 3.10.10
IPython version      : 8.13.2

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 22.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit

pandas    : 2.0.1
platform  : 1.0.8
sys       : 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
tensorflow: 2.12.0

TensorFlow version: 2.12.0
Is MPS available?: False
Using device: /device:CPU:0
Metal device set to: Apple M1 Max


In [3]:
# Using wikipedia BTC page edit history as proxy for public interest and sentiment
# - additional potential sources for 'sentiment analysis' => tweets, google trends
"""
    Downloading wikipedia edits for BTC  """

'\n    Downloading wikipedia edits for BTC  '

In [4]:
%%time
#
#  BLOCK ONE
#

import mwclient  # Module for interacting with MediaWiki API.
import time  # Module for handling time-related operations.

# Initialize a MediaWiki client site which is a class that enables work with a specific wiki site,
# in this case, the English (en) version of Wikipedia.
site = mwclient.Site('en.wikipedia.org')  # Replace 'en' with the appropriate language code for other sites.

# Specify which page to use by creating a Page object using the page's title.
page = site.pages['Bitcoin']  # Replace 'Bitcoin' with the title of alternatively desired Wikipedia page.

#

CPU times: user 42.7 ms, sys: 12.8 ms, total: 55.5 ms
Wall time: 1.24 s


In [5]:
%%time
#
#  BLOCK TWO
#

# Obtain from Wikipedia the list of revisions for the specified page.
revs = list(page.revisions())  # Returns a list of dictionaries containing information about each revision of the page.
                              # Each dictionary contains keys such as 'user', 'comment', 'timestamp', etc. that provide
                              # details about the revision. The list is sorted in reverse chronological order by default.
#

CPU times: user 1.83 s, sys: 284 ms, total: 2.11 s
Wall time: 3min 1s


In [6]:
# Look at the first set of revisions for the specified page.
# Outputs an ordered dictionary, which is a Python class that functions as a hybrid-like list and dictionary combined.
revs[0]  # Returns a dictionary containing information about the first revision of the page.
         # The dictionary contains keys such as 'user', 'comment', 'timestamp', etc. that provide details about the revision.
         # The specific information returned depends on the MediaWiki API and the parameters used to query the revisions.


OrderedDict([('revid', 1151233254),
             ('parentid', 1149274508),
             ('minor', ''),
             ('user', 'Rodw'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=4, tm_mday=22, tm_hour=18, tm_min=46, tm_sec=9, tm_wday=5, tm_yday=112, tm_isdst=-1)),
             ('comment',
              'Disambiguating links to [[Central American University]] (link changed to [[Central American University (San Salvador)]]) using [[User:Qwertyytrewqqwerty/DisamAssist|DisamAssist]].')])

In [7]:
%%time
#
#  BLOCK THREE
#

# Sort the list of revisions for the specified page in ascending order based on their timestamps.
# The sorted() function is used here with a key parameter, which takes a function that returns a value to sort by.
# In this case, the lambda function returns the timestamp value from each revision dictionary.
revs = sorted(revs, key=lambda rev: rev['timestamp'])  # Returns the same list of revision dictionaries, but sorted
                                                       # in ascending order by their timestamps.

CPU times: user 1.83 ms, sys: 1.16 ms, total: 2.99 ms
Wall time: 3.01 ms


In [8]:
# Look at the first revision in the sorted/reordered list of revisions for the specified page.
# Outputs an ordered dictionary, which is a Python class that functions as a hybrid-like list and dictionary combined.
revs[0]  
# Returns a dictionary containing information about the first revision of the page in the sorted list.

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

# Identifying the 'sentiment'of the wikipedia page edits

In [9]:
%%time
#
# BLOCK FOUR
#

# This library provides for pipelines to 'deep learning' models; requires TensorFlow 2.0 or PyTorch
# AutoModelForSequenceClassification provides a pre-trained sequence classification model
# AutoTokenizer is used to automatically select the appropriate tokenizer for a given pre-trained model. 
# The tokenizer is responsible for preprocessing raw text data into a format understood by the deep learning model.
# The pipeline function is used for running pre-trained models.
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline


# Load the pre-trained model and tokenizer
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Create a sentiment analysis pipeline using the loaded model and tokenizer
sentiment_pipeline = pipeline('sentiment-analysis',
                              model=model,            # use the loaded model
                              tokenizer=tokenizer,    # use the tokenizer
                              max_length=256,         # set the maximum input length to a smaller value (e.g., 256)
                              )

#

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


CPU times: user 691 ms, sys: 323 ms, total: 1.01 s
Wall time: 2.76 s


In [10]:
%%time
#
# BLOCK FIVE
#

#
def find_sentiment(text: str) -> float:
    """
    This function takes in a string of text and returns a sentiment score between -1 and 1,
    where negative values indicate negative sentiment and positive values indicate positive sentiment.
    """
    # run the text through the sentiment analysis model and get the first (and only) result
    sent: List[dict] = sentiment_pipeline([text[:250]])[0]
    
    # extract the sentiment score from the model output
    score: float = sent['score']
    
    # check if the sentiment label is negative
    if sent['label'] == 'NEGATIVE':
        
        # if so, multiply the score by -1 to indicate negative sentiment
        score *= -1

    # return the sentiment score    
    return score

#

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


In [11]:
# # Provides for pipeline to 'deep learning' models; requires TensorFlow 2.0 or PyTorch
# from transformers import pipeline  # import the transformers library, which includes the pipeline function for running pre-trained models

# from typing import Dict, List, Tuple, Any  # import List and Tuple types from the typing module for type hinting

# # sentiment_pipeline = pipeline("sentiment-analysis")  # initialize the Deep Learning sentiment analysis model using the pipeline function from transformers

# # initialize the Deep Learning sentiment analysis model using the pipeline function from transformers
# sentiment_pipeline = pipeline('sentiment-analysis',
#                               model='distilbert-base-uncased-finetuned-sst-2-english',
#                               max_length=256)  # reduced maximum input length to a smaller value (e.g., 256)

# def find_sentiment(text: str) -> float:
#     """
#     This function takes in a string of text and returns a sentiment score between -1 and 1,
#     where negative values indicate negative sentiment and positive values indicate positive sentiment.
#     """
#     # run the text through the sentiment analysis model and get the first (and only) result
#     sent: List[dict] = sentiment_pipeline([text[:250]])[0]
    
#     # extract the sentiment score from the model output
#     score: float = sent['score']
    
#     # check if the sentiment label is negative
#     if sent['label'] == 'NEGATIVE':
        
#         # if so, multiply the score by -1 to indicate negative sentiment
#         score *= -1

#     # return the sentiment score    
#     return score

# #

In [12]:
%%time
#
# TESTING BLOCK
#

# Test sentiment function operation
test_positive_sent = find_sentiment('I love you') 
print(test_positive_sent)

test_negative_sent = find_sentiment('I despise you') 
print(test_negative_sent)

test_neutral_sent = find_sentiment('I neutral you') 
print(test_neutral_sent)

#

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0.9998656511306763
-0.9928188920021057
-0.9936189651489258
CPU times: user 256 ms, sys: 255 ms, total: 510 ms
Wall time: 354 ms


In [13]:
#
# TESTING BLOCK
#

# Test the sentiment analysis model
# Define a list of text strings to analyze
# texts = [
#     "This is a positive sentence.",
#     "This is a negative sentence.",
#     "This is a neutral sentence.",
# ]

# Call the find_sentiment function to analyze the texts
# sentiment_scores = find_sentiment(texts)

# Print the sentiment scores
# print(sentiment_scores)

#

In [14]:
%%time
#
#  BLOCK SIX
#

#
from typing import Dict, List, Tuple, Any  # import List and Tuple types from the typing module for type hinting

# Define the 'edits' dictionary
edits: Dict[str, Dict[str, Any]] = {}

# Iterate through the revisions in the sorted list 'revs'
for rev in revs:
    # Get the date of the revision using the 'timestamp' attribute
    date: str = time.strftime('%Y-%m-%d', rev['timestamp'])
    
    # Check if the date is not in the 'edits' dictionary
    if date not in edits:
        # If the date is not in the 'edits' dictionary, create a new dictionary with the following attributes:
        # - 'sentiments': a list to store the sentiment analysis scores
        # - 'edt_count': an integer to store the number of edits made on the date
        edits[date]: Dict[str, Any] = {'sentiments': [], 'edt_count': 0}
        
    # Increment the 'edt_count' by 1 for the current date
    edits[date]['edt_count'] += 1
        
    # Get the comment for the revision, if it exists
    comment: str = rev.get('comment', '')
    
    # Call the 'find_sentiment' function to get the sentiment score for the comment
    sentiment_score: float = find_sentiment(comment)
    
    # Append the sentiment score to the 'sentiments' list for the current date
    edits[date]['sentiments'].append(sentiment_score)

#

CPU times: user 27min 54s, sys: 28min 52s, total: 56min 47s
Wall time: 33min 30s


In [15]:
%%time
#
#  BLOCK SEVEN
#

#
from statistics import mean  # import mean function from statistics module

# iterate through each key in the edits dictionary
for key in edits:
    
    # check if there are any sentiments present for the current key
    if len(edits[key]['sentiments']) > 0:
        
        # calculate the mean sentiment for the current key
        edits[key]['sentiment'] = mean(edits[key]['sentiments'])
        
        # calculate the negative sentiment score as a ratio of the total sentiment scores
        edits[key]['percent_neg_sentiment'] = len([s for s in edits[key]['sentiments'] if s < 0]) / len(edits[key]['sentiments'])
                                           
    # if there are no sentiments present for the current key
    else:
        
        # set the sentiment score to 0
        edits[key]['sentiment'] = 0
                                           
        # set the negative sentiment score to 0
        edits[key]['neg_sentiment'] = 0

        # remove the 'sentiments' key from the current key in the 'edits' dictionary
        del edits[key]['sentiments']


CPU times: user 24.9 ms, sys: 3.52 ms, total: 28.4 ms
Wall time: 66.7 ms


In [16]:
# 'edits' is Key value Pair/s dictionary, each key => date
# there are 3 key values; 'edt_count', 'sentiment', and 'neg_sentiment'
# 'edt_count' = number of times BTC wikipedia page comments was edited on a given day
# 'sentiment' = average sentiment for that day
# 'percent_neg_sentiment' = percentage of edits that express sentiment negativity on that given day

# edits

# generate a dataframe with sentiment data
transform -
structure -
clean-up dataframe

In [17]:
# Import pandas module for creating and manipulating dataframes
import pandas as pd

# Create a dataframe 'edits_df' from the dictionary 'edits' with index orientation set to 'index'
# takes in list of dictionaries, orient on 'index' ensures that each dictionary is a seperate row of the dataframe
BTC_edits_df = pd.DataFrame.from_dict(edits, orient='index')  

In [18]:

BTC_edits_df.head()

,sentiments,edt_count,sentiment,percent_neg_sentiment
2009-03-08,"[-0.9905919432640076, 0.748120903968811, -0.99...",4,-0.550525,0.75
2009-08-05,[0.748120903968811],1,0.748121,0.00
2009-08-06,"[0.995745837688446, 0.995745837688446]",2,0.995746,0.00
2009-08-14,[0.9300214052200317],1,0.930021,0.00
2009-10-13,"[0.5404372811317444, -0.9954361319541931]",2,-0.227499,0.50


In [19]:
# convert the index to a pandas 'datetime index' using pandas to_datetime() method
BTC_edits_df.index = pd.to_datetime(BTC_edits_df.index)  # transforms from string into 'datetime object' 

# Addressing the problem of gaps in dates of activity in the dataframe
there are data gaps within the dataframe

i.e. asset or security is traded more often than days of comments or sentiment generating activity.

In [20]:
#
#
#

from datetime import datetime
# from datetime import datetime

dates = pd.date_range(start= '2009-03-08', end= datetime.now())

#
#df = pd.DataFrame({'data':value}, index=pd.DatetimeIndex(timestamp), dtype=float))
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2023-04-30', '2023-05-01', '2023-05-02', '2023-05-03',
               '2023-05-04', '2023-05-05', '2023-05-06', '2023-05-07',
               '2023-05-08', '2023-05-09'],
              dtype='datetime64[ns]', length=5176, freq='D')

In [22]:
BTC_edits_df = BTC_edits_df.reindex(dates, fill_value=0)
BTC_edits_df.head()
# BTC_edits_df.tail()

,sentiments,edt_count,sentiment,percent_neg_sentiment
2009-03-08,"[-0.9905919432640076, 0.748120903968811, -0.99...",4,-0.550525,0.75
2009-03-09,0,0,0.000000,0.00
2009-03-10,0,0,0.000000,0.00
2009-03-11,0,0,0.000000,0.00
2009-03-12,0,0,0.000000,0.00


In [23]:
# create a rolling 30 day average of edits and sentiment
rolling_BTC_edits = BTC_edits_df.rolling(30).mean()

DataError: Cannot aggregate non-numeric type: object

In [ ]:
# Exploratory data anlysis
# Downloading assets/securities price data (i.e. S&P 500)

In [ ]:
# import yfinance package as package for data source
import yfinance as yf  # call yahoo finance API in order to download daily stock and indices prices

In [ ]:
# initialize a ticker class to enable download of pricing history for. single, i.e. specified symbol
sp500 = yf.Ticker('^GSPC')  # GSPC is the S&P500 index

In [ ]:
# to query historical prices
sp500 = sp500.history(period = 'max') # 'max' allows to query ALL data from beginning of index creation in dataframe format

# visualize pandas dataframe
# each row representing data of a trading day with non-trading days excluded
sp500

In [ ]:
# examine the dataframe index; note returns a 'date-time-index'
sp500.index

In [ ]:
# initiate preliminary data cleaning and visualization

In [ ]:
# visualize data frame data via plotting closing price column against the index
sp500.plot.line(y = 'Close', use_index = True)  # index values are x-axis and closing price is y-axis

In [ ]:
# eliminate columns that provide zero value for our analysis of the S&P500 index
del sp500['Dividends']
del sp500['Stock Splits']

In [ ]:
# Define target for Machine Learning, i.e. what we will actually utilize Machine Leanring to predict
# i.e. target / question is ~ will price go up or go down TOMORROW
# note, predicting direction will be both more successful and more advantagious / greater utility than attempting to predict absolute price

In [ ]:
# Create a new column in our dataframe to provide tomorrow's price
# creating new dataframe column called 'Tomorrow' 
# use pandas function '.shift()' to move the values of the 'Close' column one row ahead and 
# assign them to the 'Tomorrow' column, creating new column with the previous day's closing price
sp500['Tomorrow'] = sp500['Close'].shift(-1)  


# Visualize updated 'sp500' dataframe with new 'Tomorrow' column containing previous day's closing price.
# allowing visualization if the market has gone up or down.
sp500 


In [ ]:
# Utilizing 'Tomorrow's' price, we will define our 'Target' as a new column in the dataframe
# creating a new column named 'Target' in the dataframe.
# comparing the values of the 'Tomorrow' column to the values of the 'Close' column
# This generates a boolean (True or False) value depending on whether the 'Tomorrow' price is greater than the 'Close' price. 
sp500['Target'] = (sp500['Tomorrow'] > sp500['Close'])  

# using the pandas '.astype()' function to convert the 'Target' column boolean to integers. 
sp500['Target'] = (sp500['Tomorrow'] > sp500['Close']).astype(int)  

# Visualize updated 'sp500' dataframe with the new 'Target' column values changed from True or False to integer 1 or 0
# value of 1 means 'Tomorrow' price was greater than the 'Close' price
# value of 0 means that the 'Tomorrow' price was less than or equal to the 'Close' price.
sp500 

In [ ]:
# Clean and/or streamline data by specifying a smaller range, i.e. how far back we will go for our starting point
# Use pandas '.loc' method to specify using only those rows that are at least the specified date or later,
# Use pandas '.copy()' method method returns a copy of the DataFrame, meaning (by default) any changes made in the original DataFrame will NOT be reflected in the copy.

sp500 = sp500.loc['1990-01-01':].copy() 
# Here, we are using the pandas '.loc' method to specify that we only want to keep rows in the 'sp500' dataframe 
# that have a date of January 1st, 1990 or later. 
# We are then using the '.copy()' method to make a copy of the resulting dataframe so that any changes made to the 
# copy will not affect the original dataframe. This ensures that we are working with a clean and streamlined dataset.

# Visualize the dataframe with the defined starting row 
sp500 
# Here, we are displaying the updated 'sp500' dataframe with the defined starting row of January 1st, 1990. 
# This allows us to focus on more recent data and remove any older data that may not be relevant to our analysis.


In [ ]:
# develop initial Machine Learning Model, i.e. training a 'random forest' model
# random forest, by design tendency towards resistance to 'overfitting'
# random forest generally relatively quick in operation
# random forest can identify or recognize 'non-linear' relationships of the data which is good in modeling securities

In [ ]:
# import our first model package
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Initializing the model
# Set the random state, 'seed' to a specific value to ensure the same results are reproducible in the future
seed = 42 


# Create a Random Forest Classifier model with specified hyperparameters:
# n_estimators specify the number of random decision trees will use; generally accuracy improves with higher number of decision trees
#    setting the number of decision trees to 185, which is generally a good number for accuracy
# min_samples_split helps prevent overfitting by specifying the minimum number of samples required to split an internal node; the higher this value the LOWER the accuracy,
#    setting minimum number of samples required to split an internal node to 100, which helps to prevent overfitting.
# random_state is used to ensure the same random values are used for each run of the model for reproducibility
model = RandomForestClassifier(n_estimators=185, min_samples_split = 100, random_state = seed)

In [ ]:
# split data into training and testing datasets
# need to split taking 'time series' nature into account, as random splitting will result in leakage, i.e. using tomorrow's data to predict today
train = sp500.iloc[:-100]  # training data set - use all data rows EXCEPT for the most recent/ last 100 rows
test =  sp500.iloc[-100:]  # testing data set - use the most recent/ last 100 rows

# select the columns that will be used to train the model
predictors = ['Open', 'High', 'Low', 'Close', 'Volume']  # exclude the 'Target' and 'Tomorrow' columns to prevent leakage!

# train the model using the specified predictors and target variables
model.fit(train[predictors], train['Target'])


In [ ]:
# use trained model using the '.predict()' method passing in 'test' set with the predictors to make predictions on the testing dataset
preds = model.predict(test[predictors])

# visualize predictions (note, default will be in numpy array format) 
preds


In [ ]:
import pandas as pd
# create new pandas series with predicted values using the 'preds' array and the index from the test dataset
preds = pd.Series(preds, index = test.index)

# visualize the predictions as a pandas series
preds

In [ ]:
# assess the accuracy of the model - require metrics package/s
from sklearn.metrics import (classification_report, 
                             balanced_accuracy_score,
                             precision_score,
                             confusion_matrix,
                             roc_auc_score,
                             f1_score)

In [ ]:
# Calculating the precision score using predicted values 'preds' and actual values in the test set 'test['Target']'
precision_score_1 = precision_score(test['Target'], preds)

# visualize precision score
print(precision_score_1)

# Calculating the balanced accuracy score using predicted values 'preds' and actual values in the test set 'test['Target']'
balanced_accuracy_score_1 = balanced_accuracy_score(test['Target'], preds)

# visualize balanced accuracy score
print(balanced_accuracy_score_1)

# Calculating the confusion matrix using predicted values 'preds' and actual values in the test set 'test['Target']'
confusion_matrix_1 = confusion_matrix(test['Target'], preds)

# visualize confusion matrix
print(confusion_matrix_1)

# Calculating the classification report using predicted values 'preds' and actual values in the test set 'test['Target']'
classification_report_1 = classification_report(test['Target'], preds)

# Printing the classification report
print(classification_report_1)


In [ ]:
# concatenate the actual target values and predicted values into a new dataframe using concatenation; i.e. pd.concat()
combined = pd.concat([test['Target'], preds], axis=1)  

# visualize the combined values dataframe
print(combined)

In [ ]:
# Visualize the 'combined' dataframe using a line plot
combined.plot()

In [ ]:
# backtesting

In [ ]:
from typing import Any, List

# functionalize the prediction model
def predict(train: pd.DataFrame, test: pd.DataFrame, predictors: List[str], model: Any) -> pd.DataFrame:
    """
    Generates predictions for a test set using a trained machine learning model.
    
    Args:
        train: DataFrame of the training data, including the target variable.
        test: DataFrame of the test data.
        predictors: List of feature names to use for making predictions.
        model: A trained machine learning model object with a '.fit()' and '.predict()' method.
    
    Returns:
        A DataFrame containing the actual target values and predicted values for the test set.
    """
    
    # fit/'train' model using the specified training data and predictor variables
    model.fit(train[predictors], train['Target'])
    
    # generate predictions using the fitted model with the '.predict()' method passing in 'test' set with the predictors
    preds = model.predict(test[predictors])
    
    # convert the numpy array of predicted values to a pandas Series with the same index as the test data
    # name the Series "Predictions"
    preds = pd.Series(preds, index = test.index, name = 'Predictions') 
    
    # combine the actual target values and predicted values into a single DataFrame for comparison
    combined = pd.concat([test['Target'], preds], axis = 1)
    
    # return the DataFrame with the actual and predicted values
    return combined


In [ ]:
# Function for backtesting the model
# every trading year ~ 250 days, thus start of 2500 would be ~ 10 years of data; i.e. starts ~ 10 years back => use 10 years of data to train FIRST model
# step is 250 which is ~ 1 year, ergo train the model for one year.  So, start by using 10 years data to train the 11th year, then 11 years of data to train the 12th year, etc...

def backtest(data: pd.DataFrame, model: Any, predictors: List[str], start: int = 2500, step: int = 250) -> pd.DataFrame:
    """
    Backtests a machine learning model using rolling time windows.

    Parameters:
    -----------
    data: pd.DataFrame
        DataFrame containing stock prices and other features.
    model: Any
        Machine learning model to use.
    predictors: List[str]
        List of features to use for making predictions.
    start: int, optional
        Index of the first row to start the backtest (default is 2500).
    step: int, optional
        Number of rows to include in each test set (default is 250).

    Returns:
    --------
    pd.DataFrame
        A concatenated DataFrame of all the predictions.
    """
    
    # Create an empty list to store all the predictions
    all_predictions = []
    
    # Iterate over the specified range, starting from 'start' and stepping by 'step'; i.e. iterate through data year by year
    for i in range(start, data.shape[0], step):
        
        # Split the data into training and test sets
        train = data.iloc[0:i].copy() # The training set includes all rows up to the current index - i.e. all of years preceding current year
        test =  data.iloc[i:(i+step)].copy() # The test set includes the next 'step' rows - i.e. current year
        
        # Use the predict function to make predictions for the test set
        predictions = predict(train, test, predictors, model)
        
        # Append the predictions to the list of all predictions
        all_predictions.append(predictions)
    
    # Concatenate all the predictions into a single DataFrame and return it
    return pd.concat(all_predictions)


In [ ]:
# backtesting the model with sp500 DataFrame, the model we trained earlier, and the list of predictors
# the default start value of 2500 means that we start backtesting using data from 10 years ago
# the default step value of 250 means that we backtest using one year of data at a time
predictions = backtest(sp500, model, predictors)

In [ ]:
# examine how many times predited market would go up and how many times predicted market would go down
predictions['Predictions'].value_counts()  # '.value_counts()' will provide a numeric count for how many of each type of prediction was made

In [ ]:
# performance analytics

In [ ]:
# Calculating the precision score using predicted values 'preds' and actual values in the test set 'test['Target']'
precision_score_2 = precision_score(predictions['Target'], predictions['Predictions'])

# visualize precision score
print(precision_score_2)

# Calculating the balanced accuracy score using predicted values 'preds' and actual values in the test set 'test['Target']'
balanced_accuracy_score_2 = balanced_accuracy_score(predictions['Target'], predictions['Predictions'])

# visualize balanced accuracy score
print(balanced_accuracy_score_2)

# Calculating the confusion matrix using predicted values 'preds' and actual values in the test set 'test['Target']'
confusion_matrix_2 = confusion_matrix(predictions['Target'], predictions['Predictions'])

# visualize confusion matrix
print(confusion_matrix_2)

# Calculating the classification report using predicted values 'preds' and actual values in the test set 'test['Target']'
classification_report_2 = classification_report(predictions['Target'], predictions['Predictions'])

# Printing the classification report
print(classification_report_2)

In [ ]:
# percentage of days market actually went up
predictions['Target'].value_counts() / predictions.shape[0] # value counts of the target divided by the total number of rows

In [ ]:
# note above, actual performance of market was better than what we predicted; i.e. market rose ~53% and Machine Learning model predicted ~52%

In [ ]:
# expanding our predictors to use in model

In [ ]:
# Define a list of horizons in days that will be used to calculate rolling averages of the S&P 500 stock index.
# The list includes values of 2, 5, 60, 250, and 1000, which correspond to different periods of time: two days, one trading week (5 days), 
# three trading months (60 days), one trading year (250 days), and four trading years (1000 days).
horizons = [2, 5, 60, 250, 1000]

# Create an empty list to store the new predictor columns to be added to the dataset.
new_predictors = []

# Loop through each horizon value in the horizons list.
for horizon in horizons:
    # Calculate the rolling average of the S&P 500 stock index for the current horizon value.
    rolling_averages = sp500.rolling(horizon).mean()
    
    # Create a new column name for the ratio of the closing price to the rolling average for the current horizon value.
    ratio_column = f'Close_Ratio_{horizon}'
    
    # Calculate the ratio of the closing price to the rolling average for the current horizon value, 
    # and add it as a new column to the S&P 500 dataset.
    sp500[ratio_column] = sp500['Close'] / rolling_averages['Close']
    
    # Create a new column name for the trend over the current horizon value.
    trend_column = f'Trend_{horizon}'
    
    # Calculate the trend over the current horizon value by taking the sum of the 'Target' column over the previous horizon period 
    # and add it as a new column to the S&P 500 dataset.
    # The shift(1) method shifts the data by 1 period so that the sum will be taken over the previous horizon period.
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()['Target']
    
    # Add the new ratio and trend column names to the new_predictors list.
    new_predictors += [ratio_column, trend_column]


In [ ]:
# Drop any rows in the `sp500` DataFrame that contain missing values (i.e., NaN values)
sp500 = sp500.dropna()

# Visualize the modified `sp500` DataFrame that has had missing value rows removed. 
# This allows us to inspect the remaining data and ensure that it is complete and appropriate for analysis.
sp500


In [ ]:
# Model improvement and optimizing - second model

In [ ]:
# re-initialize our model (change model parameters)
# set random state, i.e. seed to specific value to ensure repeatability given specified start point of randomness
# n_estimators specify the number of random decision trees will use; generally accuracy improves with higher number of decision trees
# min_samples_split aids in further preventing 'overfitting'; the higher this value the lower the accuracy, however the higher value will decrease risk of overfit

seed = 42
model = RandomForestClassifier(n_estimators=200, min_samples_split = 50, random_state = seed)

In [ ]:
# Update predict function; the prediction model

def predict(train: pd.DataFrame, test: pd.DataFrame, predictors: List[str], model: Any) -> pd.DataFrame:
    """
    Train the specified model using the specified training data and predictor variables, and use the model to predict
    the values of the 'Target' variable for the specified test data.
    
    Parameters:
    train (pd.DataFrame): The training data used to fit the model.
    test (pd.DataFrame): The test data used to generate predictions.
    predictors (List[str]): A list of column names to use as predictor variables.
    model (Any): The model to train and use for predictions.
    
    Returns:
    pd.DataFrame: A DataFrame containing the actual and predicted values of the 'Target' variable.
    """
    
    # Train the model using the specified 'predictors' columns, then attempt to predict the 'Target'
    model.fit(train[predictors], train['Target'])
    
    # Generate predictions with the model using the '.predict()' method passing in 'test' set with the predictors.
    # Predictions are probabilities for the class with the highest probability.
    # Extract the probabilities for the positive class, which is column 1 of the predictions array.
    preds = model.predict_proba(test[predictors])[:,1]
    
    # Convert predicted probabilities to binary predictions:
    # Set predictions >= 0.6 to 1, indicating an expected price increase
    # Set predictions < 0.6 to 0, indicating an expected price decrease or no change
    # these values will reduce number of actual trding days, thereby increase the chance or probability that day of trade price goes up
    preds[preds >= 0.6] = 1
    preds[preds < 0.6] = 0
    
    # Turn the numpy array into a pandas series and name it 'Predictions'
    preds = pd.Series(preds, index = test.index, name = 'Predictions') 
    
    # Generate a dataframe of actual values (i.e. test['Target']) and predicted values ('preds') concatenation
    combined = pd.concat([test['Target'], preds], axis = 1)
    
    return combined


In [ ]:
# backtesting the S&P 500 data with the earlier created model and asociated predictors created earlier.
predictions = backtest(sp500, model, new_predictors)

In [ ]:
predictions['Predictions'].value_counts()

In [ ]:
# Calculating the precision score using predicted values 'preds' and actual values in the test set 'test['Target']'
precision_score_3 = precision_score(predictions['Target'], predictions['Predictions'])

# visualize precision score
print(precision_score_3)

# Calculating the balanced accuracy score using predicted values 'preds' and actual values in the test set 'test['Target']'
balanced_accuracy_score_3 = balanced_accuracy_score(predictions['Target'], predictions['Predictions'])

# visualize balanced accuracy score
print(balanced_accuracy_score_3)

# Calculating the confusion matrix using predicted values 'preds' and actual values in the test set 'test['Target']'
confusion_matrix_3 = confusion_matrix(predictions['Target'], predictions['Predictions'])

# visualize confusion matrix
print(confusion_matrix_3)

# Calculating the classification report using predicted values 'preds' and actual values in the test set 'test['Target']'
classification_report_3 = classification_report(predictions['Target'], predictions['Predictions'])

# Printing the classification report
print(classification_report_3)